In [25]:
import pandas as pd
import re
from tqdm import tqdm
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import numpy as np

In [142]:
path ="/Users/michaelmiller/Documents/GitHub/He_who_laughs_last/data/Russian_Anecdotes/lemmatized_russian.csv"

jokes = pd.read_csv(path)

In [143]:
def preprocess_read_russia(jk):
    res = []
    for i in jk:
        tmp = []
        i = str(i).replace('[','')
        i = str(i).replace(',','')
        i = str(i).replace(']','')
        i = str(i).replace('ё','е')
        i = str(i).replace("'",'')
        i = str(i).replace("©",'')
        i = str(i).replace("'",'')
        i = str(i).replace("–", '')
        i = str(i).replace("\\", '')
        i = str(i).replace("»", '')
        i = str(i).replace("«", '')
        i = str(i).replace("…", '')
        for k in i.split():
            tmp.append(k)
        res.append(tmp)
    return res
        

In [144]:
jokes['lemmatized'] = preprocess_read_russia(jokes['lemmatized'])

In [89]:


patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.replace('.', ' ')
            token = token.replace('"', ' ')
            token = token.replace("'", ' ')
            token = token.replace("ё", 'е')
            token = token.replace("»", ' ')
            token = token.replace("©", ' ')
            token = token.replace("–", ' ')
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None


In [145]:
# lemm = []
# for i in tqdm(jokes['0']):
#     lemm.append(lemmatize(i))

In [114]:
def load_dict(path):
    slovarik = np.load(path, allow_pickle = True)
    slovarik = slovarik.tolist()
    return slovarik

In [115]:
dct = load_dict("/Users/michaelmiller/Documents/GitHub/He_who_laughs_last/data/SVD/russian_nofraglit_SVD_dict.npy")

In [134]:
def lemmatized_to_coords(text,slovarik):
    coords = []
    unk = {}
    cnt=0
    for i in text:
        if(str(i)!="nan"):
            temp=[]
            for j in i:
                try:
                    temp.append(list(slovarik[j]))
                except KeyError:
                    if j not in unk.keys():
                        unk[j]=1
                    else: unk[j]+=1
            coords.append(temp)
        else: coords.append([])
   
    return unk

In [146]:
coords = lemmatized_to_coords(jokes,dct)

In [58]:
type(dct['0'])

numpy.ndarray

In [147]:
coords = sorted(coords.items(), key=lambda x:x[1],reverse=True)

In [141]:
sum(x[1] for x in coords)

362881

In [148]:
coords

[('сша', 4734),
 ('секс', 2891),
 ('выборы', 2617),
 ('интернет', 2491),
 ('рф', 2315),
 ('коронавирус', 2172),
 ('трамп', 2096),
 ('навальный', 1994),
 ('госдума', 1493),
 ('порошенко', 1349),
 ('байден', 1319),
 ('обама', 1243),
 ('коррупция', 1066),
 ('лукашенко', 1021),
 ('полищук', 985),
 ('изя', 897),
 ('евро', 873),
 ('вирус', 867),
 ('компьютер', 846),
 ('сталин', 714),
 ('фсб', 696),
 ('nan', 688),
 ('ковид', 670),
 ('штирлиц', 657),
 ('сирия', 636),
 ('млрд', 627),
 ('айфон', 605),
 ('турция', 605),
 ('чубайс', 602),
 ('беларусь', 590),
 ('янукович', 589),
 ('евросоюз', 580),
 ('сериал', 566),
 ('менеджер', 545),
 ('яндекс', 545),
 ('рейтинг', 525),
 ('жириновский', 523),
 ('офис', 519),
 ('смартфон', 517),
 ('пандемия', 512),
 ('собянин', 512),
 ('газпром', 504),
 ('мид', 502),
 ('меркель', 500),
 ('собчак', 493),
 ('дебил', 490),
 ('вакцинация', 478),
 ('самоизоляция', 477),
 ('шойгу', 456),
 ('видео', 444),
 ('анатолиевич', 443),
 ('№', 441),
 ('хоккей', 435),
 ('ввп', 429

In [150]:

.to_csv(path, index=False)

KeyboardInterrupt: 

In [151]:
jokes =pd.DataFrame(jokes)

KeyboardInterrupt: 